In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\nsthtz\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %                  ]  53.8 %         ]  77.1 % Resolving package versions...


In [111]:
using DelimitedFiles
data = readdlm(joinpath("dummyMajor.csv"),';')
#data = readdlm(joinpath("dummyMinor.csv"),';')
data = round.(Int, data)

128×24 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮         

# # Convert piano roll into Tx12 matrix for chord enforcement


In [112]:
#

timestep_chords = zeros(Int8, size(data, 2), 12)
i=1
data[:, 1]

for i in 1:size(data, 2)
    active_notes = findall(x->x==1, data[:,i])
    #println(active_notes)
    for note in active_notes
        timestep_chords[i, note%12+1] = 1
    end
    #println(timestep_chords[i,:])
end

timestep_chords

24×12 Array{Int8,2}:
 0  0  0  0  0  0  0  0  0  0  1  0
 1  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  0

# # Pattern matcher to identify potential Major or Minor roots

In [126]:
#Default: C Major

pattern = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]
majorpattern = [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
minorpattern = [0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0]

Keys = Dict{Integer, String}(1 => "C", 2 => "C#", 3 => "D", 4 => "D#", 5 => "E", 6 => "F", 7 => "F#", 8 => "G", 9 => "G#", 10 => "A", 11 => "A#", 12 => "B")


#Score In: Tx12 Matrix, pattern. Out: Best position(s)
function score(matrix, pattern)
    counter = zeros(Int8, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    println("Counter is ", counter)
    
    score = zeros(Int8, 12, 1)
    
    for i in 1:size(pattern, 1)
        for j in 1:size(pattern, 1)
            score[i] += circshift(pattern, i-1)[j]*counter[j]
        end
    end
    println("Score is ", score)
    return score
end

score (generic function with 1 method)

In [123]:
# Find maximum score function
function findmaximum(score_result)
    positions = []
    max_value = maximum(score_result)
    for (i, x) in enumerate(score_result)
        if x == max_value
            push!(positions, i)
        end
    end
    #positions = [i for (i, x) in enumerate(score_result) if x == max_value]
    return positions
end

findmaximum (generic function with 1 method)

In [136]:
# Score matrix after pattern
score_result = score(timestep_chords, pattern)
# score_result = [9, 5, 18, 9, 23, 18, 8, 2, 7, 16, 12, 11]

# Find positions of maximum score
positions = findmaximum(score_result)

#TODO: Use positions to find the best key

if size(positions, 1) > 1
    
else
    tonality_score = []   #[Major, Minor] after count of roots, fourths and fifths
    push!(tonality_score, score_result)
end
    
#Save Key
best_key = maximum(tonality_score)
key = Keys[best_key]
    
# Shift matrix to key with best score
timestep_chords_shifted = timestep_chords

timestep_chords_shifted = circshift(timestep_chords, (0,-(best_key-1)))

for i in 1:size(timestep_chords_shifted, 1)
    println(timestep_chords_shifted[i,:])
end




Counter is Int8[2 0 2 5 0 3 0 3 0 3 6 0]
Score is Int8[13; 16; 8; 21; 8; 19; 14; 10; 19; 5; 24; 11]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0

Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# # Integer Program for Fifth enforcement

In [80]:
using JuMP, Ipopt
#model = Model(with_optimizer(GLPK.Optimizer))
#@variable()
#@constraint()
#@objective(model,,)

┌ Info: Recompiling stale cache file C:\Users\nsthtz\.julia\compiled\v1.1\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1184
┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1186
